<h1><center>Exportación de datos usando Python</center></h1> 
<h2><center>Autor: Marina Montecchiarini</center></h2>


## Proceso

- Se armó un prototipo de archivo de Excel, llamado "Datos experimentos LAB", en el cual se insertaron resultados ficticios de un laboratorio. Cada vez que se hace el experimento se arma una solapa nueva en ese archivo y ciertas celdas de Excel se modifican para ingresar los nuevos datos del experimento.
- En base a este archivo se desea armar uno nuevo: "Resultados LAB" para poder analizar los resultados. Idealmente ese se debería actualizar cada semana con los nuevos resultados obtenidos.
- Se trabaja con Pandas y con openpyxl.



In [1]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\marina\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [14]:


import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

def read_and_extract_data(file_path, sheet_name):
    # Leer la hoja de Excel
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Extraer los datos específicos
    fecha = df.iat[0, 1]          # B1
    tecnico = df.iat[0, 5]        # F1
    temperatura = df.iat[6, 3]    # D7
    hora = df.iat[16, 1]          # B17
    valor_obtenido = df.iat[16, 2] # C17
    equipo = df.iat[11, 6]        # G12
    observacion = df.iat[25, 1]   # B26
    
    # Crear un diccionario con los datos extraídos
    data = {
        'Fecha': fecha,
        'Tecnico': tecnico,
        'Temperatura': temperatura,
        'Hora': hora,
        'Valor Obtenido': valor_obtenido,
        'Equipo': equipo,
        'Observacion': observacion
    }
    
    return data

def combine_sheets(file_path):
    # Leer el archivo de Excel
    xls = pd.ExcelFile(file_path)
    
    # Lista para almacenar los datos extraídos de cada hoja
    data_list = []
    
    # Iterar sobre los nombres de las hojas
    for sheet_name in xls.sheet_names:
        data = read_and_extract_data(file_path, sheet_name)
        data_list.append(data)
    
    # Crear un DataFrame con los datos combinados
    combined_df = pd.DataFrame(data_list)
    
    return combined_df

def create_plot(df, output_image_path):
    # Crear el gráfico
    plt.figure(figsize=(10, 6))
    plt.scatter(df['Fecha'], df['Valor Obtenido'], color='black')  # Gráfico de puntos en color negro
    plt.xlabel('Fecha')
    plt.ylabel('Valor Obtenido')
    plt.title('Valores Obtenidos vs Fecha')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Guardar el gráfico como imagen
    plt.savefig(output_image_path)
    plt.close()

def insert_image_into_excel(excel_file_path, image_path):
    # Cargar el archivo de Excel
    wb = load_workbook(excel_file_path)
    
    # Crear una nueva hoja para el gráfico
    if 'Grafico' not in wb.sheetnames:
        ws = wb.create_sheet(title='Grafico')
    else:
        ws = wb['Grafico']
    
    # Insertar la imagen en la nueva hoja
    img = Image(image_path)
    ws.add_image(img, 'A1')
    
    # Guardar el archivo de Excel
    wb.save(excel_file_path)

def main():
    # Ruta del archivo de Excel de origen
    input_file_path = 'C:/Users/Marina/Portfolio/Datos experimentos LAB.xlsx'
    
    # Combinar las hojas
    combined_df = combine_sheets(input_file_path)
    
    # Mostrar las primeras filas del DataFrame combinado
    print(combined_df.head())
    
    # Guardar el DataFrame combinado en un nuevo archivo de Excel
    output_file_path = 'C:/Users/Marina/Portfolio/Resultados LAB.xlsx'
    combined_df.to_excel(output_file_path, index=False)
    
    # Crear y guardar el gráfico
    output_image_path = 'grafico_valores_obtenidos_vs_fecha.png'
    create_plot(combined_df, output_image_path)
    
    # Insertar el gráfico en el archivo de Excel
    insert_image_into_excel(output_file_path, output_image_path)

if __name__ == "__main__":
    main()

       Fecha Tecnico  Temperatura      Hora  Valor Obtenido         Equipo  \
0 2022-01-05  BB, BB           37  12:00:00            1.00  Laboratorio X   
1 2023-07-28  AA, AA           39  12:10:00            1.01  Laboratorio X   
2 2022-11-03  AA, AA           37  12:00:00            1.03  Laboratorio H   
3 2023-06-03  AA, AA           39  12:00:00            1.00  Laboratorio X   
4 2023-11-16  BB, BB           39  12:05:00            1.00  Laboratorio X   

           Observacion  
0        color extraño  
1  mucha contaminación  
2    sin observaciones  
3    sin observaciones  
4        color extraño  
